# **Modeling and Evaluation Notebook**

## Objectives

- Answer Business Requirement 2: Develop a Machine Learning model to classify cherry leaves as Healthy or Infected, enabling the prediction of powdery mildew presence.
- Build and evaluate models: Implement a baseline CNN model, refine it through hyperparameter tuning, and assess performance.
- Analyze model effectiveness: Use Saliency Maps and t-SNE visualization to interpret model predictions and feature separability.
- Compare model performance: Select the best model based on accuracy, loss, and efficiency for real-world deployment.
- Prepare for deployment: Save the optimized model for integration into a Streamlit web app hosted on Heroku.

## Inputs

Dataset
- inputs/mildew_dataset/cherry-leaves/train
- inputs/mildew_dataset/cherry-leaves/validation
- inputs/mildew_dataset/cherry-leaves/test

Precomputed Features (from Data Visualization Notebook)
- Image Shape Standardization → 128x128x3 for consistency across models.
- Class Distribution Analysis → Ensures balanced dataset splits.
- Feature Space Visualization → PCA confirms class separability.
- Pixel Intensity Distribution → Confirms brightness variations relevant for classification.

## Outputs

Data Processing & Visualization
- Dataset Distribution Plot → Confirmed balanced data split across training, validation, and test sets.
- Data Augmentation Visualization → Showcased applied transformations, including rotation, flipping, and zooming.
Model Training & Optimization
- Baseline CNN Model → Implemented a standard CNN to establish initial performance.
- Hyperparameter-Tuned CNN → Optimized model performance using Keras Tuner (adjusting filters, dropout, learning rate, and L2 regularization).
- Best Model Selection → Chose the Tuned CNN based on test accuracy and generalization ability.
- Saved Trained Models → Final model stored for Streamlit integration and deployment on Heroku.
Model Evaluation & Explainability
- Learning Curves → Visualized loss and accuracy trends over epochs.
- Confusion Matrices → Displayed classification performance for train, validation, and test sets.
- Classification Reports → Provided precision, recall, and F1-score analysis.
- Saliency Maps → Highlighted regions in images that influenced the model’s predictions.
- Feature Space Visualization (t-SNE) → Compared Baseline CNN and Tuned CNN feature separability.

## Additional Comments

- Business Impact: The trained model can assist in early detection of powdery mildew, reducing manual inspection time and improving plantation monitoring efficiency.
- Data-Driven Enhancements: Model improvements were guided by data preprocessing insights, including PCA feature visualization and class balance validation.
- Deployment Readiness: The best model was optimized and prepared for integration into a Streamlit web app for real-world application.



---

# Set Data Directory

---

## Import Necessary Packages

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

## Set Working Directory

In [ ]:
cwd= os.getcwd()

In [ ]:
os.chdir('/workspaces/mildew-detector')
print("You set a new current directory")

#### Confirm the new current directory

In [ ]:
work_dir = os.getcwd()
work_dir

## Set Input Directories

In [ ]:
# Set input directories
my_data_dir = 'inputs/mildew_dataset/cherry-leaves'
train_path = os.path.join(my_data_dir, 'train')
val_path = os.path.join(my_data_dir, 'validation')
test_path = os.path.join(my_data_dir, 'test')

## Set Output Directory

In [ ]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

## Set Label Names

In [ ]:
# Set the labels for the images
labels = os.listdir(train_path)
print('Label for the images are', labels)

## Set Image Shape

In [ ]:
## Import saved image shape embedding
import joblib
version = 'v1'
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

---

## Number of Images in Train, Test and Validation Data

In [ ]:
import os
import pandas as pd

# Define dataset path
my_data_dir = "inputs/mildew_dataset/cherry-leaves"

# Define dataset splits and labels
sets = ['train', 'test', 'validation']
labels = ['Healthy', 'Infected']

# Create a dictionary for structured storage
data = {
    'Set': [],
    'Label': [],
    'Frequency': []
}

total_images = 0  # Initialize total count

# Loop through each dataset split and count images
for set_name in sets:
    for label in labels:
        path = os.path.join(my_data_dir, set_name, label)
        num_images = len(os.listdir(path)) if os.path.exists(path) else 0  
        
        # Print count
        print(f"{set_name}/{label}: {num_images} images")

        # Store in dictionary
        data['Set'].append(set_name)
        data['Label'].append(label)
        data['Frequency'].append(num_images)

        # Update total count
        total_images += num_images

# Convert dictionary to DataFrame
df_freq = pd.DataFrame(data)

# Print total images
print(f"\nTotal number of images: {total_images}")

# Display DataFrame
import ace_tools as tools
tools.display_dataframe_to_user(name="Image Frequency Data", dataframe=df_freq)

### Bar Chart - Image Distribution

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(8, 5))
sns.barplot(data=df_freq, x='Set', y='Frequency', hue='Label')
plt.title("Image Distribution in Dataset")
plt.xlabel("Dataset Split")
plt.ylabel("Number of Images")
plt.savefig(f'{file_path}/labels_distribution.png', bbox_inches='tight', dpi=150)
plt.show()

---

# Implement Data Augmentation

---

### ImageDataGenerator

In [ ]:
# Import TensorFlow/Keras ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Augment Training, Validation, and Test Sets

- Initialize ImageDataGenerator for Data Augmentation

In [ ]:
# Define Augmentation for Training Set
augmented_image_data = ImageDataGenerator(rotation_range=20,
                                          width_shift_range=0.10,
                                          height_shift_range=0.10,
                                          shear_range=0.1,
                                          zoom_range=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='nearest',
                                          rescale=1./255
                                          )

- Augment Training Image Dataset

In [ ]:
batch_size = 20  # Set batch size
train_set = augmented_image_data.flow_from_directory(train_path,
                                                     target_size=image_shape[:2],
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='binary',
                                                     shuffle=True
                                                     )

train_set.class_indices

- Augment Validation Image Dataset

In [ ]:
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                        target_size=image_shape[:2],
                                                                        color_mode='rgb',
                                                                        batch_size=batch_size,
                                                                        class_mode='binary',
                                                                        shuffle=False
                                                                        )

validation_set.class_indices

- Augment Test Image Dataset

In [ ]:
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                  target_size=image_shape[:2],
                                                                  color_mode='rgb',
                                                                  batch_size=batch_size,
                                                                  class_mode='binary',
                                                                  shuffle=False
                                                                  )

test_set.class_indices

---

## Visualization of Augmented Images

### Plot Augmented Training Image

In [ ]:
for _ in range(3):
    img, label = next(train_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

### Plot Augmented Validation and Test Images

In [ ]:
for _ in range(3):
    img, label = next(validation_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

In [ ]:
for _ in range(3):
    img, label = next(test_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

### Save Class Indices

In [ ]:
joblib.dump(value=train_set.class_indices,
            filename=f"{file_path}/class_indices.pkl")

### Compare Multiple Augmented Images in a Grid

In [ ]:
def plot_augmented_images_grid(data_generator, num_images=10):
    """Displays a grid of augmented images to visualize transformation effects."""
    img_batch, label_batch = next(data_generator)

    fig, axes = plt.subplots(2, num_images // 2, figsize=(15, 6))
    
    for i in range(num_images):
        ax = axes[i // (num_images // 2), i % (num_images // 2)]
        ax.imshow(img_batch[i])
        ax.axis("off")

    plt.suptitle("Augmented Image Variations (Training Set)")
    plt.show()

# Display the augmented image grid
plot_augmented_images_grid(train_set)

---

# Model Creation

---

### Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input  
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

## Base Convolutional Neural Network (CNN)

For this project, a Convolutional Neural Network (CNN) was selected because it is highly effective for image classification tasks. Unlike traditional machine learning models, CNNs can automatically learn hierarchical spatial features from images, making them ideal for detecting powdery mildew in cherry leaves.

### Why CNN Over Other Models?

| **Model**                        | **Reason for Exclusion** |
|----------------------------------|-----------------------------------------------------------|
| **Logistic Regression**          | Inefficient for image data; lacks feature extraction capabilities. |
| **Random Forest**                | Performs well on structured data but struggles with high-dimensional images. |
| **Support Vector Machines (SVM)** | Computationally expensive for large image datasets; lacks spatial feature extraction. |
| **Fully Connected Neural Networks (MLP)** | Requires excessive parameters and lacks spatial feature learning. |

CNNs are specifically designed for image processing, as they utilize convolutional layers to detect local features like **leaf texture, shape, and mildew presence** while minimizing computational overhead. Unlike traditional machine learning models, CNNs leverage **spatial hierarchies** in images, enabling efficient feature extraction without requiring extensive manual preprocessing.

### Key Features of the Model
- **Four convolutional layers** efficiently extract hierarchical patterns.
- **Batch normalization** stabilizes training and accelerates convergence.
- **L2 regularization (0.001)** prevents overfitting by constraining large weights.
- **Dropout (0.3)** improves generalization by reducing reliance on specific neurons.
- **Adam optimizer (0.0001 LR)** balances training speed and stability.
- **Sigmoid activation** ensures a probability-based classification of "Healthy" or "Infected".

In [ ]:
def create_base_cnn():
    """
    Creates a Convolutional Neural Network (CNN) model for binary classification.

    The model consists of:
    - Convolutional layers with ReLU activation and L2 regularization
    - MaxPooling layers for downsampling
    - Fully connected Dense layers with Dropout for regularization
    - Sigmoid activation for binary classification (Healthy vs. Infected)

    Returns:
        model: A compiled Keras CNN model
    """

    model = Sequential(
        [
            # Explicit Input Layer (Fixes the Warning)
            Input(shape=image_shape),  

            # First convolutional block
            Conv2D(
                filters=32,
                kernel_size=(3, 3),
                activation="relu",
                kernel_regularizer=l2(0.001),  
            ),
            MaxPooling2D(pool_size=(2, 2)),

            # Second convolutional block
            Conv2D(
                filters=64,
                kernel_size=(3, 3),
                activation="relu",
                kernel_regularizer=l2(0.001),
            ),
            MaxPooling2D(pool_size=(2, 2)),

            # Third convolutional block
            Conv2D(
                filters=128,
                kernel_size=(3, 3),
                activation="relu",
                kernel_regularizer=l2(0.001),
            ),
            MaxPooling2D(pool_size=(2, 2)),

            # Fourth convolutional block
            Conv2D(
                filters=128,
                kernel_size=(3, 3),
                activation="relu",
                kernel_regularizer=l2(0.001),
            ),
            MaxPooling2D(pool_size=(2, 2)),

            # Flatten the feature maps into a single vector
            Flatten(),

            # Fully connected layers
            Dense(128, activation="relu", kernel_regularizer=l2(0.001)),
            Dropout(0.3),  # Dropout layer to prevent overfitting

            # Output layer for binary classification
            Dense(1, activation="sigmoid"),
        ]
    )

    # Compile Model
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

# Instantiate the CNN model
model_cnn = create_base_cnn()

### Model Summary 

In [ ]:
# Define the base CNN model
model_base_cnn = create_base_cnn()

# Print model summary
model_base_cnn.summary()

---

## Model Training

### Early Stopping Implementation

In [ ]:
# Import required callbacks
from tensorflow.keras.callbacks import EarlyStopping

# Set EarlyStopping callback
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

### Fit CNN model for training

In [ ]:
# Create the CNN model
model_base_cnn = create_base_cnn()

# Train the base CNN model
history_base_cnn = model_base_cnn.fit(
    train_set,
    epochs=20,
    steps_per_epoch=len(train_set.classes) // batch_size,
    validation_data=validation_set,
    callbacks=[early_stop],
    verbose=1
)

### Save the CNN Model

In [ ]:
# Save the trained base CNN model
model_base_cnn.save('outputs/v1/mildew_detector_base_cnn.keras')
print("Base CNN model saved successfully!")

---

# Model Performance & Evaluation

---

## Evaluate Base CNN on the Test Set

In [ ]:
# Evaluate the base CNN model
test_loss_base_cnn, test_accuracy_base_cnn = model_base_cnn.evaluate(test_set)

# Print evaluation results
print(f"Test Accuracy (Base CNN): {test_accuracy_base_cnn:.4f}")
print(f"Test Loss (Base CNN): {test_loss_base_cnn:.4f}")

## Save Training History for Base CNN

In [ ]:
# Convert training history of Base CNN to DataFrame
import pandas as pd

df_history_base_cnn = pd.DataFrame(history_base_cnn.history)

# Save history for later use
df_history_base_cnn.to_csv("outputs/v1/history_base_cnn.csv", index=False)
print("Base CNN training history saved.")

## Model Learning Curve (Loss & Accuracy) for Base CNN

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set Seaborn style
sns.set_style("whitegrid")

# Plot Loss Curve
df_history_base_cnn[["loss", "val_loss"]].plot(style=".-")
plt.title("Loss")
plt.legend(["Training Loss", "Validation Loss"])
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{file_path}/model_training_losses.png", bbox_inches="tight", dpi=150)
plt.show()

# Plot Accuracy Curve
df_history_base_cnn[["accuracy", "val_accuracy"]].plot(style=".-")
plt.title("Accuracy")
plt.legend(["Training Accuracy", "Validation Accuracy"])
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{file_path}/model_training_acc.png", bbox_inches="tight", dpi=150)
plt.show()

Remarks:

The learning curve indicates a smooth decline in training and validation loss, showing that the model is learning effectively. However, the slight divergence at later epochs suggests potential overfitting. The accuracy curve stabilizes close to 100%, implying strong model performance.

## Confusion Matrix & Classification Report (Train & Test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Ensure output directory exists
output_dir = "outputs/v1"
os.makedirs(output_dir, exist_ok=True)


def generate_confusion_matrix(y_true, y_pred, label_map, set_name):
    """
    Generates, displays, and saves a static confusion matrix.
    
    Parameters:
    - y_true: Actual class labels
    - y_pred: Predicted class labels
    - label_map: List of class names
    - set_name: Dataset name (Train, Validation, Test)
    """
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=label_map, columns=label_map)

    # Plot confusion matrix
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues", linewidths=0.5)
    plt.title(f"Confusion Matrix - {set_name} Set")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.tight_layout()

    # Save and show confusion matrix
    save_path = os.path.join(output_dir, f"confusion_matrix_{set_name.lower()}.png")
    plt.savefig(save_path, dpi=150)
    plt.show()  # Display in the notebook
    print(f"Confusion Matrix saved at: {save_path}")


def generate_classification_report(y_true, y_pred, label_map, set_name):
    """
    Generates, prints, and saves a classification report.
    
    Parameters:
    - y_true: Actual class labels
    - y_pred: Predicted class labels
    - label_map: List of class names
    - set_name: Dataset name (Train, Validation, Test)
    """
    report = classification_report(y_true, y_pred, target_names=label_map)
    
    print(f"\n--- Classification Report: {set_name} Set ---\n")
    print(report)

    # Save report as a text file
    report_path = os.path.join(output_dir, f"classification_report_{set_name.lower()}.txt")
    with open(report_path, "w") as f:
        f.write(report)

    print(f"Classification report saved at: {report_path}")


def evaluate_model(generator, model, label_map, set_name, threshold=0.5):
    """
    Evaluates model performance by generating a confusion matrix and classification report.
    
    Parameters:
    - generator: Data generator (train, validation, or test)
    - model: Trained model
    - label_map: List of class names
    - set_name: Dataset name (Train, Validation, Test)
    - threshold: Probability threshold for classification (default: 0.5)
    """
    y_true = generator.classes  # True labels
    y_pred_probs = model.predict(generator)  # Model predictions (probabilities)
    y_pred = (y_pred_probs > threshold).astype(int).flatten()  # Convert to class labels

    print(f"\n#### {set_name} Set Evaluation ####\n")

    # Generate and display confusion matrix
    generate_confusion_matrix(y_true, y_pred, label_map, set_name)

    # Generate and print classification report
    generate_classification_report(y_true, y_pred, label_map, set_name)


# Get class labels from training set
label_map = list(train_set.class_indices.keys())

# Evaluate the model on Train, Validation, and Test sets
evaluate_model(train_set, model_base_cnn, label_map, "Train")
evaluate_model(test_set, model_base_cnn, label_map, "Test")

**Remarks:**

- Train Set: The model exhibits moderate misclassification between healthy and infected leaves, with precision and recall around 52%. This suggests the model struggles to distinguish between the two classes in the training set.

- Test Set: The model performs significantly better, achieving near-perfect classification (precision, recall, and F1-score of 1.00). This indicates the model generalizes well, despite inconsistencies in the training set.

### Load Saved Model (必要？？？)

In [ ]:
from keras.models import load_model
model = load_model('outputs/v1/mildew_detector_cnn.keras')
print("\nModel loaded successfully!")

## Save Evaluation Pickle

In [ ]:
# Store evaluation results with the correct variable names
evaluation = {
    "test_loss": test_loss_base_cnn, 
    "test_accuracy": test_accuracy_base_cnn
}

import joblib

# Save evaluation results
joblib.dump(value=evaluation, filename="outputs/v1/evaluation_base_cnn.pkl")
print("\nModel evaluation results saved!")

---

# Detecting and Analyzing Overfitting

---

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert training history to DataFrame (Base CNN)
df_history_base_cnn = pd.DataFrame(history_base_cnn.history)  # Ensure correct variable is used

# Set style for plots
sns.set_style("whitegrid")

# Plot Training & Validation Loss
plt.figure(figsize=(8, 4))
plt.plot(df_history_base_cnn["loss"], label="Training Loss", marker="o")
plt.plot(df_history_base_cnn["val_loss"], label="Validation Loss", marker="o")
plt.title("Loss Curve (Base CNN)")
plt.legend()
plt.grid(True)
plt.show()

# Plot Training & Validation Accuracy
plt.figure(figsize=(8, 4))
plt.plot(df_history_base_cnn["accuracy"], label="Training Accuracy", marker="o")
plt.plot(df_history_base_cnn["val_accuracy"], label="Validation Accuracy", marker="o")
plt.title("Accuracy Curve (Base CNN)")
plt.legend()
plt.grid(True)
plt.show()

# Identify Overfitting
train_acc = df_history_base_cnn["accuracy"].iloc[-1]
val_acc = df_history_base_cnn["val_accuracy"].iloc[-1]
train_loss = df_history_base_cnn["loss"].iloc[-1]
val_loss = df_history_base_cnn["val_loss"].iloc[-1]

print("\n### Overfitting Analysis (Base CNN) ###")
print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Validation Accuracy: {val_acc:.4f}")
print(f"Final Training Loss: {train_loss:.4f}")
print(f"Final Validation Loss: {val_loss:.4f}")

# Improved Overfitting Check
if (train_acc - val_acc > 0.05) or (train_loss < val_loss - 0.05):
    print("\nWarning: Possible Overfitting Detected!")
else:
    print("\nNo significant overfitting detected. The model generalizes well.")

**Remark: Overfitting Analysis (Base CNN)**

The training and validation curves show consistent trends, indicating that the model generalizes well without significant overfitting.

Key Observations:
- Training vs. Validation Accuracy: Both curves closely follow each other, with final validation accuracy (0.9762) slightly higher than training accuracy (0.9500).
- Training vs. Validation Loss: The loss values remain similar, suggesting that the model is not memorizing the training data but learning meaningful patterns.
- No sharp divergence between training and validation metrics, reinforcing that overfitting is not a major concern.

Conclusion: The base CNN demonstrates good generalization, making it a reliable baseline for further improvements. 

---

# Hyperparameter Tuning with Keras Tuner

---

Optimize the CNN model using Keras Tuner to improve performance while preventing overfitting.

### Import Necessary Libraries

In [ ]:
pip install keras-tuner

In [ ]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input,  
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

## Define the Hyperparameter Search Space
The hyperparameter values were selected based on a balance between model performance, generalization ability, and computational efficiency. The choices aim to enhance feature extraction while preventing overfitting and maintaining training stability.

| **Hyperparameter**       | **Values Tested**       | **Rationale** |
|-------------------------|------------------------|--------------|
| **Number of Filters** (num_filters_1 to num_filters_4) | [32, 64, 128, 256] | Increasing filter sizes in deeper layers helps extract hierarchical features while balancing computational cost. |
| **L2 Regularization** (l2_reg) | [0.0001, 0.001, 0.0005] | Helps prevent overfitting by applying weight penalties, ensuring smooth generalization. |
| **Dropout Rate** (dropout_rate) | [0.2, 0.3] | Reduces overfitting by randomly deactivating neurons during training. Moderate dropout rates retain learning capacity while preventing memorization. |
| **Learning Rate** (learning_rate) | [0.0001, 0.0003]| Chosen for stable convergence. Lower rates avoid overshooting minima, while slightly higher rates accelerate learning without destabilization. |

The tuning process balances model complexity and generalization, ensuring optimal feature extraction without overfitting or excessive computational burden. 

In [ ]:
# Define image shape globally
image_shape = (128, 128, 3) 

# Function to define CNN model with hyperparameter tuning
def build_model(hp):
    """
    Define a CNN model with hyperparameter tuning using Keras Tuner.

    Parameters:
    - hp: Keras Tuner search space

    Returns:
    - Compiled Keras model
    """
    model = Sequential(
        [
            # Explicit Input Layer
            Input(shape=image_shape),

            # First convolutional block
            Conv2D(
                filters=hp.Choice("num_filters_1", values=[32, 64]),
                kernel_size=(3, 3),
                activation="relu",
                kernel_regularizer=l2(hp.Choice("l2_reg", values=[0.0001, 0.001])),
            ),
            MaxPooling2D(pool_size=(2, 2)),

            # Second convolutional block
            Conv2D(
                filters=hp.Choice("num_filters_2", values=[64, 128]),
                kernel_size=(3, 3),
                activation="relu",
                kernel_regularizer=l2(hp.Choice("l2_reg", values=[0.0001, 0.001])),
            ),
            MaxPooling2D(pool_size=(2, 2)),

            # Third convolutional block
            Conv2D(
                filters=hp.Choice("num_filters_3", values=[128, 256]),
                kernel_size=(3, 3),
                activation="relu",
                kernel_regularizer=l2(hp.Choice("l2_reg", values=[0.0001, 0.001])),
            ),
            MaxPooling2D(pool_size=(2, 2)),

            # Fourth convolutional block
            Conv2D(
                filters=hp.Choice("num_filters_4", values=[128, 256]),
                kernel_size=(3, 3),
                activation="relu",
                kernel_regularizer=l2(hp.Choice("l2_reg", values=[0.0001, 0.001])),
            ),
            MaxPooling2D(pool_size=(2, 2)),

            # Flatten Layer
            Flatten(),

            # Fully connected layer with regularization
            Dense(128, activation="relu", kernel_regularizer=l2(hp.Choice("l2_reg", values=[0.0001, 0.0005]))),
            Dropout(hp.Choice("dropout_rate", values=[0.2, 0.3])),

            # Output layer for binary classification
            Dense(1, activation="sigmoid"),
        ]
    )

    # Compile Model
    model.compile(
        optimizer=Adam(
            learning_rate=hp.Choice("learning_rate", values=[0.0001, 0.0003])
        ),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

## Run Hyperparameter Search

Search for the best hyperparameters using Keras Tuner’s RandomSearch.

In [ ]:
# Define the tuner
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",  # Optimize for highest validation accuracy
    max_trials=7,  # Limits the number of model variations
    executions_per_trial=1,  # Runs each model once
    directory="keras_tuner_results",
    project_name="cnn_tuning",
)

# Run hyperparameter search
tuner.search(
    train_set,
    validation_data=validation_set,
    epochs=8,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)],  
)

## Retrieve Best Hyperparameters & Print Model Summary

In [ ]:
# Retrieve best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\n### Best Hyperparameters Found ###")
print(f"Filters (Conv1): {best_hps.get('num_filters_1')}")
print(f"Filters (Conv2): {best_hps.get('num_filters_2')}")
print(f"Filters (Conv3): {best_hps.get('num_filters_3')}")
print(f"Filters (Conv4): {best_hps.get('num_filters_4')}")
print(f"L2 Regularization (Conv1): {best_hps.get('l2_reg')}")
print(f"Dropout Rate: {best_hps.get('dropout_rate')}")
print(f"Learning Rate: {best_hps.get('learning_rate')}")

# Build the best model with selected hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Print the model summary
best_model.summary()

## Train the Best Model & Store History

In [ ]:
history_tuned_cnn = best_model.fit(
    train_set,
    validation_data=validation_set,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)],  
    verbose=1,
)

## Save the Best Tuned Model

In [ ]:
best_model.save("outputs/v1/mildew_detector_cnn_tuned.keras")
print("\nBest tuned CNN model saved successfully!")

---

# Tuned Model Evaluation

---

## Evaluate the Tuned Model

In [ ]:
# Load & Evaluate the Tuned Model
best_model = tf.keras.models.load_model("outputs/v1/mildew_detector_cnn_tuned.keras")

test_loss_tuned, test_accuracy_tuned = best_model.evaluate(test_set)

print("\n### Tuned Model Evaluation ###")
print(f"Test Accuracy: {test_accuracy_tuned:.4f}")
print(f"Test Loss: {test_loss_tuned:.4f}")

## Save Training History

In [ ]:
# Convert Training History to DataFrame (Tuned CNN)
import pandas as pd

df_history_tuned_cnn = pd.DataFrame(history_tuned_cnn.history)

# Save history for later use
df_history_tuned_cnn.to_csv("outputs/v1/history_tuned_cnn.csv", index=False)
print("Tuned CNN training history saved.")

## Model Learning Curve (Loss & Accuracy) for Tuned CNN

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set Seaborn style
sns.set_style("whitegrid")

# Plot Loss Curve
df_history_tuned_cnn[["loss", "val_loss"]].plot(style=".-")
plt.title("Loss")
plt.legend(["Training Loss", "Validation Loss"])
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{file_path}/model_training_losses.png", bbox_inches="tight", dpi=150)
plt.show()

# Plot Accuracy Curve
df_history_tuned_cnn[["accuracy", "val_accuracy"]].plot(style=".-")
plt.title("Accuracy")
plt.legend(["Training Accuracy", "Validation Accuracy"])
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{file_path}/model_training_acc.png", bbox_inches="tight", dpi=150)
plt.show()

## Confusion Matrix & Classification Report for Tuned CNN

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

# Ensure output directory exists
output_dir = "outputs/v1"
os.makedirs(output_dir, exist_ok=True)

# Function to Generate Confusion Matrix
def generate_confusion_matrix(y_true, y_pred, label_map, set_name):
    """
    Generates, displays, and saves a static confusion matrix.
    
    Parameters:
    - y_true: Actual class labels
    - y_pred: Predicted class labels
    - label_map: List of class names
    - set_name: Dataset name (Train, Validation, Test)
    """
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=label_map, columns=label_map)

    # Plot confusion matrix
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues", linewidths=0.5)
    plt.title(f"Confusion Matrix - {set_name} Set")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.tight_layout()

    # Save and show confusion matrix
    save_path = os.path.join(output_dir, f"confusion_matrix_{set_name.lower()}.png")
    plt.savefig(save_path, dpi=150)
    plt.show()  # Display in the notebook
    print(f"Confusion Matrix saved at: {save_path}")

# Function to Generate Classification Report
def generate_classification_report(y_true, y_pred, label_map, set_name):
    """
    Generates, prints, and saves a classification report.
    
    Parameters:
    - y_true: Actual class labels
    - y_pred: Predicted class labels
    - label_map: List of class names
    - set_name: Dataset name (Train, Validation, Test)
    """
    report = classification_report(y_true, y_pred, target_names=label_map)
    
    print(f"\n--- Classification Report: {set_name} Set ---\n")
    print(report)

    # Save report as a text file
    report_path = os.path.join(output_dir, f"classification_report_{set_name.lower()}.txt")
    with open(report_path, "w") as f:
        f.write(report)

    print(f"Classification report saved at: {report_path}")

# Function to Evaluate the Model
def evaluate_model(generator, model, label_map, set_name, threshold=0.5):
    """
    Evaluates model performance by generating a confusion matrix and classification report.
    
    Parameters:
    - generator: Data generator (train, validation, or test)
    - model: Trained model
    - label_map: List of class names
    - set_name: Dataset name (Train, Validation, Test)
    - threshold: Probability threshold for classification (default: 0.5)
    """
    y_true = generator.classes  # True labels
    y_pred_probs = model.predict(generator)  # Model predictions (probabilities)
    y_pred = (y_pred_probs > threshold).astype(int).flatten()  # Convert to class labels

    print(f"\n#### {set_name} Set Evaluation ####\n")

    # Generate and display confusion matrix
    generate_confusion_matrix(y_true, y_pred, label_map, set_name)

    # Generate and print classification report
    generate_classification_report(y_true, y_pred, label_map, set_name)

# Load the Tuned CNN Model
tuned_model_path = "outputs/v1/mildew_detector_cnn_tuned.keras"
tuned_model = load_model(tuned_model_path)  

# Get class labels from training set
label_map = list(train_set.class_indices.keys())

# Evaluate the Tuned Model on Train and Test Sets
evaluate_model(train_set, tuned_model, label_map, "Train")
evaluate_model(test_set, tuned_model, label_map, "Test")

### **Tuned CNN Model Evaluation Summary**

#### **Training Set Performance**
- **Accuracy:** 52%  
- **Precision, Recall, F1-score:** 0.52 for both Healthy and Infected classes  
- **Key Insight:** The training performance suggests that the model may not have fully learned class distinctions, as accuracy is close to random guessing.

#### **Test Set Performance**
- **Accuracy:** 100%  
- **Precision, Recall, F1-score:** 1.00 for both Healthy and Infected classes  
- **Key Insight:** The model perfectly classifies test samples, indicating **overfitting** on the training data.

#### **Possible Concern**
- The extreme difference in accuracy between training (52%) and test (100%) suggests **potential overfitting**, meaning the model might have memorized the test data rather than generalizing well.
- Further analysis, such as **cross-validation or additional regularization**, may be necessary to ensure the model's robustness.

**Conclusion:** While the Tuned CNN performs **perfectly on test data**, its poor training accuracy signals the need for further investigation to prevent overfitting. 

---

## Compare Models (Base CNN vs Tuned CNN)

### Plot Accuracy & Loss Bar Chart

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Ensure correct variable names are used
models = ["Base CNN", "Tuned CNN"]
accuracy_values = [test_accuracy_base_cnn, test_accuracy_tuned]
loss_values = [test_loss_base_cnn, test_loss_tuned]

# Function to add value labels
def add_value_labels(ax, values):
    for i, v in enumerate(values):
        ax.text(i, v + 0.02, f"{v:.4f}", ha="center", fontsize=12, fontweight="bold")

# Plot Accuracy Comparison
plt.figure(figsize=(8, 5))
ax = plt.bar(models, accuracy_values, color=["blue", "green"])
plt.ylabel("Test Accuracy")
plt.title("Accuracy Comparison: Base CNN vs. Tuned CNN")
plt.ylim(0, 1)  # Ensure accuracy is within [0,1]
plt.grid(axis="y", linestyle="--", alpha=0.7)
add_value_labels(plt.gca(), accuracy_values)
plt.show()

# Plot Loss Comparison
plt.figure(figsize=(8, 5))
ax = plt.bar(models, loss_values, color=["red", "purple"])
plt.ylabel("Test Loss")
plt.title("Loss Comparison: Base CNN vs. Tuned CNN")
plt.grid(axis="y", linestyle="--", alpha=0.7)
add_value_labels(plt.gca(), loss_values)
plt.show()

### **Model Performance Comparison**
#### **Accuracy Comparison**
- **Base CNN:** Achieved lower test accuracy.  
- **Tuned CNN:** Showed a significant accuracy improvement, indicating better feature extraction and generalization.  

#### **Loss Comparison**
- **Base CNN:** Higher test loss, suggesting inefficient learning and potential misclassifications.  
- **Tuned CNN:** Drastically lower test loss, reinforcing that the model optimally fits the data with improved robustness.  

 **Conclusion:** The **Tuned CNN outperforms the Base CNN**, demonstrating **higher accuracy and lower loss**, making it a more reliable choice for real-world deployment.  

---

# Model Explainability & Feature Space Visualization

---

## t-SNE Plot for Comparing Feature Separability of Base CNN and Tuned CNN

This t-SNE visualization represents the feature space of the test set as extracted from the Base CNN and Tuned CNN before classification.

In [ ]:
# Step 1: Import Required Libraries
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Model, load_model

# Step 2: Load Both Models
base_model_path = "outputs/v1/mildew_detector_cnn.keras"
tuned_model_path = "outputs/v1/mildew_detector_cnn_tuned.keras"

base_model = load_model(base_model_path)
tuned_model = load_model(tuned_model_path)

# Function to Extract Features and Apply t-SNE
def extract_tsne_features(model, test_set, model_name):
    feature_extractor = Model(
        inputs=model.layers[0].input, 
        outputs=model.get_layer(index=-2).output  # Extract from second-last layer
    )
    X_test_features = feature_extractor.predict(test_set)
    y_test_labels = test_set.classes  

    # Adjust Perplexity
    num_samples = X_test_features.shape[0]
    adjusted_perplexity = min(30, num_samples - 1)

    # Apply t-SNE
    tsne = TSNE(n_components=2, perplexity=adjusted_perplexity, random_state=42)
    X_tsne = tsne.fit_transform(X_test_features)

    # Convert to DataFrame
    df_tsne = pd.DataFrame(X_tsne, columns=["t-SNE1", "t-SNE2"])
    df_tsne["Label"] = y_test_labels  

    # Define Class Mapping
    label_map = {v: k for k, v in train_set.class_indices.items()}  
    df_tsne["Label"] = df_tsne["Label"].map(label_map)

    return df_tsne

# Step 3: Extract Features for Both Models
df_tsne_base = extract_tsne_features(base_model, test_set, "Base CNN")
df_tsne_tuned = extract_tsne_features(tuned_model, test_set, "Tuned CNN")

# Step 4: Plot t-SNE Results (Side-by-Side)
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# t-SNE for Base CNN
sns.scatterplot(ax=axes[0], data=df_tsne_base, x="t-SNE1", y="t-SNE2", hue="Label", palette=["blue", "orange"], alpha=0.7)
axes[0].set_title("t-SNE: Base CNN")
axes[0].grid(True)

# t-SNE for Tuned CNN (Different Color Scheme)
sns.scatterplot(ax=axes[1], data=df_tsne_tuned, x="t-SNE1", y="t-SNE2", hue="Label", palette=["green", "red"], alpha=0.7)
axes[1].set_title("t-SNE: Tuned CNN")
axes[1].grid(True)

plt.tight_layout()
plt.show()

**Remark**

The t-SNE visualization highlights the improvement in feature separability after hyperparameter tuning.
- Base CNN: Shows partial separation between Healthy and Infected samples but with noticeable overlap, indicating difficulty in distinguishing the classes.
- Tuned CNN: Demonstrates clearer feature clustering, suggesting enhanced feature extraction and improved classification performance.

Conclusion: The Tuned CNN extracts more distinguishable features, contributing to its higher accuracy compared to the Base CNN. 

---

## Saliency Maps for Tuned CNN

Saliency maps help visualize which parts of an image were most influential in the model’s classification decision.

In [ ]:
# Import Required Libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2  # OpenCV for better heatmap visualization

# Load the Tuned CNN Model
tuned_model_path = "outputs/v1/mildew_detector_cnn_tuned.keras"
tuned_model = tf.keras.models.load_model(tuned_model_path)

# Define a Function to Compute and Enhance Saliency Maps
def compute_saliency_map(model, image, class_index):
    """
    Computes a Saliency Map to highlight important pixels influencing the model's decision.

    Parameters:
    - model: Trained CNN model
    - image: Input image (single sample)
    - class_index: Target class index (0 for Healthy, 1 for Infected)

    Returns:
    - Enhanced saliency map as a NumPy array
    """
    image = tf.convert_to_tensor(image[None], dtype=tf.float32)  # Add batch dimension

    with tf.GradientTape() as tape:
        tape.watch(image)
        preds = model(image)
        loss = preds[:, class_index]  # Focus on the target class

    grads = tape.gradient(loss, image)[0]  # Compute gradients
    saliency = np.max(np.abs(grads), axis=-1)  # Take max across color channels

    # Normalize Saliency Map
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-8)  # Normalize between 0-1

    # Apply Exponential Scaling to Enhance Weak Signals
    saliency = np.power(saliency, 3)  # Amplify small differences

    # Convert to 8-bit (0-255) for Better Visualization
    saliency = (saliency * 255).astype(np.uint8)

    # Apply Histogram Equalization to Improve Contrast
    saliency = cv2.equalizeHist(saliency)

    # Apply a Stronger Colormap for More Visible Heatmap
    saliency_colored = cv2.applyColorMap(saliency, cv2.COLORMAP_HOT)  # Use 'HOT' colormap for high contrast

    return saliency_colored

# Select a Test Image
sample_index = 7  # Adjust if needed
X_test_batch, y_test_batch = next(iter(test_set))  # Extract a batch
sample_image = X_test_batch[sample_index]  # Select one test image
sample_label = int(y_test_batch[sample_index])  # Convert label to integer

# Compute Saliency Map
saliency = compute_saliency_map(tuned_model, sample_image, class_index=sample_label)

# Display the Original Image and Enhanced Saliency Map
plt.figure(figsize=(10, 5))

# Show Original Image 
plt.subplot(1, 2, 1)
plt.imshow(np.clip(sample_image * 255, 0, 255).astype("uint8"))  
plt.axis("off")
plt.title("Original Image (Enhanced Contrast)")

# Show Strongly Enhanced Saliency Map
plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(saliency, cv2.COLOR_BGR2RGB))  # Convert OpenCV BGR to RGB
plt.axis("off")
plt.title("Strongly Enhanced Saliency Map")

plt.show()

**Remark**

The Saliency Map highlights the most influential regions that guided the CNN’s classification decisions.

- Bright yellow/orange areas indicate the regions the model prioritized for classification.
- The model focuses on specific patterns and textures rather than random noise, confirming its ability to extract meaningful features.
- Saliency regions vary across images, suggesting that the CNN adapts dynamically based on the input.

This analysis reinforces the model’s interpretability, demonstrating that its predictions are based on relevant visual features. 

---

# Final Model Selection & Evaluation

---

## Load and Compare Models

### Load Both Models

In [ ]:
tuned_cnn_model = load_model("outputs/v1/mildew_detector_cnn_tuned.keras")
print("Tuned CNN Model Loaded Successfully.")

In [ ]:
from tensorflow.keras.models import load_model
import pandas as pd

# Load the base CNN model
base_cnn_model = load_model("outputs/v1/mildew_detector_base_cnn.keras")

# Load the best tuned CNN model
tuned_cnn_model = load_model("outputs/v1/mildew_detector_tuned_cnn.keras")

# Load training history DataFrames
df_history_base_cnn = pd.read_csv("outputs/v1/history_base_cnn.csv")
df_history_tuned_cnn = pd.read_csv("outputs/v1/history_tuned_cnn.csv")

print("Both models and their training histories have been loaded successfully!")

### Evaluate Both Models On the Test Set

In [ ]:
# Evaluate both models
base_test_loss, base_test_accuracy = base_cnn_model.evaluate(test_set, verbose=0)
tuned_test_loss, tuned_test_accuracy = tuned_cnn_model.evaluate(test_set, verbose=0)

# Print evaluation results
print("\n### Base CNN Evaluation ###")
print(f"Test Accuracy: {base_test_accuracy:.4f}")
print(f"Test Loss: {base_test_loss:.4f}")

print("\n### Tuned CNN Evaluation ###")
print(f"Test Accuracy: {tuned_test_accuracy:.4f}")
print(f"Test Loss: {tuned_test_loss:.4f}")

### Compare Model Performance in a Structured Table

In [ ]:
import pandas as pd
from IPython.display import display 

# Create the comparison dataframe
comparison_df = pd.DataFrame(
    {
        "Model": ["Base CNN", "Tuned CNN"],
        "Test Accuracy": [base_test_accuracy, tuned_test_accuracy],
        "Test Loss": [base_test_loss, tuned_test_loss],
    }
)

# Display results
display(comparison_df)

### Select the Best Model:

In [ ]:
# Select the best model: prioritize accuracy first, then lower loss as tiebreaker
if tuned_test_accuracy > base_test_accuracy:
    best_model_name = "Tuned CNN"
elif tuned_test_accuracy == base_test_accuracy and tuned_test_loss < base_test_loss:
    best_model_name = "Tuned CNN"
else:
    best_model_name = "Base CNN"

# Assign correct model path
best_model_path = (
    "outputs/v1/mildew_detector_cnn_tuned.keras"
    if best_model_name == "Tuned CNN"
    else "outputs/v1/mildew_detector_base_cnn.keras"
)

# Print final selection
print("\n### Best Model Selected ###")
print(f"Model: {best_model_name}")
print(f"Test Accuracy: {tuned_test_accuracy:.4f} (Tuned) | {base_test_accuracy:.4f} (Base)")
print(f"Test Loss: {tuned_test_loss:.4f} (Tuned) | {base_test_loss:.4f} (Base)")

---

# Assess Model Efficiency

---

## Model Complexity Analysis

In [ ]:
import tensorflow as tf

# Get model parameter counts
base_cnn_params = base_cnn_model.count_params() / 1e6  # Convert to millions
tuned_cnn_params = tuned_cnn_model.count_params() / 1e6  # Convert to millions

# Define Model Names and Parameter Counts
models = ["Base CNN", "Tuned CNN"]
params = [base_cnn_params, tuned_cnn_params]

# Plot Model Complexity (Parameter Count)
plt.figure(figsize=(8, 5))
plt.bar(models, params, color=["blue", "orange"])
plt.ylabel("Parameter Count (Millions)")
plt.title("Model Complexity: Parameter Count Comparison")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

## Compare Model Sizes

In [ ]:
import matplotlib.pyplot as plt
import os

# Get model sizes (in MB)
base_cnn_size = os.path.getsize("outputs/v1/mildew_detector_base_cnn.keras") / (1024 * 1024)
tuned_cnn_size = os.path.getsize("outputs/v1/mildew_detector_cnn_tuned.keras") / (1024 * 1024)

# Define Model Names and Sizes
models = ["Base CNN", "Tuned CNN"]
sizes = [base_cnn_size, tuned_cnn_size]

# Plot Model Size Comparison
plt.figure(figsize=(8, 5))
plt.bar(models, sizes, color=["blue", "orange"])
plt.ylabel("Model Size (MB)")
plt.title("Model Storage Comparison")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

## Computational Performance Cost

In [ ]:
import time

# Measure Inference Time
def measure_inference_time(model, test_set):
    sample_input, _ = next(iter(test_set))  # Get one batch
    start_time = time.time()
    _ = model.predict(sample_input)
    end_time = time.time()
    return end_time - start_time  # Return execution time

# Compute Inference Time
base_cnn_time = measure_inference_time(base_cnn_model, test_set)
tuned_cnn_time = measure_inference_time(tuned_cnn_model, test_set)

# Define Model Names and Inference Times
models = ["Base CNN", "Tuned CNN"]
times = [base_cnn_time, tuned_cnn_time]

# Plot Computation Time Comparison
plt.figure(figsize=(8, 5))
plt.bar(models, times, color=["blue", "orange"])
plt.ylabel("Inference Time (Seconds per Batch)")
plt.title("Computation Time Comparison")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

---

# Check Business Case Requirement

---

In [ ]:
# Define the required accuracy threshold (example: 90%)
required_accuracy = 0.90  

# Evaluate final model
final_model = load_model(best_model_path)
final_test_loss, final_test_accuracy = final_model.evaluate(test_set, verbose=0)

# Print Performance Evaluation Summary
print("\n### Business Case Requirement Check ###")
print(f"Required Accuracy: {required_accuracy * 100:.2f}%")
print(f"Final Model Achieved Accuracy: {final_test_accuracy * 100:.2f}%")

# Decision
if final_test_accuracy >= required_accuracy:
    print("\nThe model meets the business performance requirement.")
else:
    print("\nThe model does NOT meet the business requirement. Consider further tuning.")

---

# 	Final Model Selection

---

## Summary of Model Selection & Evaluation (Written Explanation)

## Save the Best Model for Deployment

In [ ]:
# Save the best model as final for deployment
final_model_path = "outputs/v1/final_mildew_detector.keras"
final_model.save(final_model_path)

print("\n### Best Model Saved for Deployment ###")
print(f"Model: {best_model_name}")
print(f"Saved at: {final_model_path}")

## Select & Display a Sample Image

In [ ]:
# Function to select and display a sample image
def load_sample_image(test_set, sample_idx=0):
    """
    Select a sample image from the test set and display it.
    """
    test_images, test_labels = next(iter(test_set))  # Get batch of images
    sample_image = test_images[sample_idx]  # Select one image
    sample_label = test_labels[sample_idx]  # Get its label

    # Display the image
    plt.imshow(sample_image)
    plt.axis("off")
    plt.title(f"Sample Image - {'Healthy' if sample_label == 0 else 'Infected'}")
    plt.show()

    return sample_image, sample_label


# Load and display a sample image from the test set
sample_image, sample_label = load_sample_image(test_set, sample_idx=0)

---

# Predict on New Images

---

## Load & Predict on Sample Image

Load test images and classify them using the trained model.

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the final model
model = load_model("outputs/v1/final_mildew_detector.keras")

# Select an image by specifying its index (pointer)
pointer = 60
label = labels[1]  # Selecting an 'Infected' leaf image

# Load the selected image and resize it
pil_image = image.load_img(
    test_path + "/" + label + "/" + os.listdir(test_path + "/" + label)[pointer],
    target_size=image_shape,
    color_mode="rgb",
)

# Convert the image to an array and normalize it
my_image = image.img_to_array(pil_image) / 255.0
my_image = np.expand_dims(my_image, axis=0)  # Add batch dimension

# Make a prediction
pred_proba = model.predict(my_image)[0, 0]  # Extract prediction probability

# Map indices to class labels
target_map = {v: k for k, v in train_set.class_indices.items()}  # Reverse mapping
pred_class = target_map[int(pred_proba > 0.5)]  # **Fixed: Ensure correct mapping**

# Adjust probability if needed
if pred_class == target_map[0]:
    pred_proba = 1 - pred_proba

# Display results
print(f"Image shape: {pil_image.size}")
print(f"Image mode: {pil_image.mode}")
print(f"Predicted class: {pred_class}")
print(f"Prediction probability: {pred_proba:.4f}")

# Show the image
pil_image

In [ ]:
# Define a list of pointers
pointers = [10, 30, 50, 70]
label = labels[1]  # 'Infected' or 'Healthy'

fig, axes = plt.subplots(1, len(pointers), figsize=(15, 5))

for i, pointer in enumerate(pointers):
    img_list = os.listdir(test_path + "/" + label)

    if pointer >= len(img_list):
        print(f"Skipping pointer {pointer}, index out of range.")
        continue

    img_path = test_path + "/" + label + "/" + img_list[pointer]

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    pred = model.predict(img_array)[0, 0]
    pred_class = "Healthy" if pred < 0.5 else "Infected"

    # Plot the image and prediction result
    axes[i].imshow(img)
    axes[i].set_title(f"{pred_class}\nProb: {pred:.4f}")
    axes[i].axis("off")

plt.tight_layout()
plt.show()

---

# Conclusion and Next Steps

---

In this project, we successfully developed a deep learning model to detect Powdery Mildew on Cherry Leaves using a structured, beginner-friendly approach.

## Key Achievements
- Baseline CNN Implementation → Developed an initial CNN model to establish a performance benchmark.
- Optimized Hyperparameter Tuning → Applied systematic tuning (adjusting filters, dropout, learning rate, and L2 regularization) to enhance model performance while balancing computational efficiency.
- Model Evaluation & Comparison → Assessed the baseline and optimized CNN models based on accuracy, loss, and generalization ability.
- Explainability with Saliency Maps → Visualized important regions influencing the model’s predictions, enhancing interpretability.
- Final Model Selection → The Hyperparameter-Tuned CNN was chosen based on its superior accuracy and robustness for real-world deployment.

## Next Steps: Model Deployment

The next step is to integrate the optimized CNN model into a user-friendly application that allows real-time classification of leaf images.

Deployment Plan
- Develop an Interactive Web App → Implement a Streamlit-based interface where users can upload leaf images for classification.
- Integrate the Tuned CNN Model → Load the trained model to process new images and predict mildew presence.
- Deploy on a Cloud Platform → Host the web application using Streamlit and Heroku for accessibility.

This deployment will enable real-time detection of powdery mildew, aiding efficient disease monitoring and automated plantation management.
